In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def logic_win(x,x_r):
    return (x >= x_r[0]) & (x < x_r[1])

#### Import and clean dataset, select out London

In [7]:
path = '../Data/UK/combined_collisions.csv'

df = pd.read_csv(path)
df.drop(columns='Unnamed: 0',inplace=True)

In [8]:
#Time column has some nans in it, must drop those entries

def is_str(x):
    Nx = len(x)
    I = np.ones(Nx).astype(bool)
    for ii in range(Nx):
        if type(x[ii]) != str:
            I[ii] = False
    return I

df = df.loc[is_str(df.Time.values),:]
df.aadf_Count_point_id = df.aadf_Count_point_id.values.astype(int)

In [25]:
#make hour, year and month columns
    #hour will be rounded 
def get_hour(T):
    Nt = len(T)
    H = np.zeros(Nt).astype(int)
    for ii in range(Nt):
        s = T[ii].split(':')
        h = int(s[0])
        m = int(s[1])
        if m > 30:
            h += 1
        H[ii] = h % 24
    return H

def get_ymd(D):
    Nd = len(D)
    # |
    ymd = np.zeros((Nd,3)).astype(int)
    for ii in range(Nd):
        s = D[ii].split('-')
        for nn in range(3):
            ymd[ii,nn] = int(s[nn])
    return ymd

def reformat_aadf_year(Year):
    Ny = len(Year)
    y = np.zeros(Ny).astype(int)
    for ii in range(Ny):
        y[ii] = int(Year[ii].split('-')[0])
    return y

df[['y','m','d']] = get_ymd(df.Date.values)
df['h'] = get_hour(df.Time.values)
df['aadf_Year'] = reformat_aadf_year(df.aadf_Year.values)

AttributeError: 'float' object has no attribute 'split'

In [26]:
Year = df.aadf_Year.values
Ny = len(Year)
y = np.zeros(Ny).astype(int)
for ii in range(Ny):
    y[ii] = int(Year[ii].split('-')[0])


AttributeError: 'float' object has no attribute 'split'

In [37]:
df.aadf_Year.sample(20)

230598    2017-01-01 00:00:00
85767     2010-01-01 00:00:00
204133                    NaN
35673     2007-01-01 00:00:00
14114                     NaN
235090    2018-01-01 00:00:00
152053                    NaN
15608                     NaN
209769                    NaN
240038                    NaN
8469                      NaN
122926    2012-01-01 00:00:00
75377                     NaN
225427                    NaN
2392      2005-01-01 00:00:00
90786                     NaN
182481    2015-01-01 00:00:00
210048                    NaN
84068     2010-01-01 00:00:00
215672                    NaN
Name: aadf_Year, dtype: object

In [10]:
#central London box
Lat_r = [51.475,51.543]
Lon_r = [-0.179,0.023]

I_lat = (df.Latitude > Lat_r[0]) & (df.Latitude < Lat_r[1])
I_lon = (df.Longitude > Lon_r[0]) & (df.Longitude < Lon_r[1])
I = I_lat & I_lon

df_lond = df.loc[I,:].copy()

In [11]:
#drop all data that don't have traffic matches

I = df_lond.match == True
df_lond = df_lond.loc[I,:]

#### Make neighborhoods feature

In [12]:
#Make a 3 (Lat, N/S) x 6 (Lon, E/W) grid
Nlat = 3
Nlon = 6
Nneigh = Nlon*Nlat

edges_lat = np.linspace(*Lat_r,Nlat+1)
dlat = np.diff(edges_lat)[0]
bins_lat = edges_lat[:-1]+0.5*dlat
edges_lon = np.linspace(*Lon_r,Nlon+1)
dlon = np.diff(edges_lon)[0]
bins_lon = edges_lon[:-1]+0.5*dlon

coord_neigh = [None]*Nneigh

df_lond['neigh'] = 0 #initialize neighborhood feature
ir = 0 #these are lat bins (rows)
ic = 0 #lon bins
for ii in range(Nneigh):
    Ilat = logic_win(df_lond.Latitude,(edges_lat[ir],edges_lat[ir+1]))
    Ilon = logic_win(df_lond.Longitude,(edges_lon[ic],edges_lon[ic+1]))
    I = Ilat & Ilon
    df_lond.loc[I,'neigh'] = ii
    
    coord_neigh[ii] = np.array([bins_lat[ir],bins_lon[ic]])
    
    ic+=1
    if ic == Nlon:
        ic = 0
        ir+=1

#### Make a DataFrame using df_lond that has entries only for traffic IDs. Main feature is # of incidents.¶
- Other features are the aadf features that go along with the ID
- Also maybe some averaged features from accidents dataset (like junction control, maybe just use the classification with the most entries for that ID)
- Maybe the target variable should be number of incidents normalized by the bike volume (this is prob. of an incident)

##### Features to include
- all aadf features (do they neeed to be averaged over year? Maybe just average anyway)
- modes of classification features in accidents
- num. of incidents
- num. of serious and fatal incidents
- num. of incidents normalized to bike volume from aadf

In [22]:
ID = traf_IDs[3]
I = df_lond.aadf_Count_point_id == ID
df_lond.loc[I,'aadf_Year'].value_counts()

2014-01-01 00:00:00    19
2013-01-01 00:00:00    14
2008-01-01 00:00:00    13
2012-01-01 00:00:00    13
2017-01-01 00:00:00    13
2006-01-01 00:00:00    12
2010-01-01 00:00:00    12
2011-01-01 00:00:00    11
2007-01-01 00:00:00    10
2015-01-01 00:00:00     9
2016-01-01 00:00:00     9
2018-01-01 00:00:00     9
2009-01-01 00:00:00     8
2005-01-01 00:00:00     7
Name: aadf_Year, dtype: int64

In [14]:
traf_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().index)
Ninc_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().values)

I = Ninc_IDs > 10 #this will select roads to include in the new DataFrame based on if they had at least this many accidents
I.sum()

IDs_df = traf_IDs[I]
Nid = len(IDs_df)

df_road = pd.DataFrame({'aadfID':IDs_df})
slight_rd = np.zeros(Nid).astype(int)
ser_rd = np.zeros(Nid).astype(int)
total_rd = np.zeros(Nid).astype(int)
for nn,ii in enumerate(IDs_df):
    I = df_lond.aadf_Count_point_id.values == ii
    total_rd[nn] = I.sum()
    slight_rd[nn] = df_lond.loc[I,'slight'].values.sum()
    ser_rd[nn] = total_rd[nn] - slight_rd[nn]
df_road['total'] = total_rd
df_road['slight'] = slight_rd
df_road['serious'] = ser_rd

safety_mode = ['neigh','Road_name','Road_name2','h','Day_of_Week','m','Speed_limit','Junction_Detail','Junction_Control','Pedestrian_Crossing-Physical_Facilities']
aadf_av = ['aadf_Pedal_cycles','aadf_Pedal_cycles','aadf_All_motor_vehicles','aadf_All_HGVs','aadf_LGVs','aadf_Buses_and_coaches']
aadf_mode = ['aadf_Road_name','aadf_Road_category','aadf_Road_type']
for f in safety_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]

for f in aadf_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]
        
for f in aadf_av:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mean()
        
df_road['total_norm'] = df_road['total'].values/df_road['aadf_Pedal_cycles'].values
df_road['serious_ratio'] = df_road['serious'].values/df_road['total'].values